# ===========================================================
# Full pipeline for the influence matrix estimation problem on the supervised dataset from the Jeopardy-like logs for comparison on different models
# ===========================================================

Goals:
1. Split the data into test and train, and validation for multiple runs
2. Formulate all different models of convex optimization, neural networks, and tower models.
3. Give the same splits to all models, tune the hyperparameters with validation set, and report train and test erros as a pickle, a table, and a figure

#### Started on: 30 Dec 2019
#### Last update: 02 Jan 2020

# Imports

In [1]:
from __future__ import division, print_function, absolute_import, unicode_literals

import imp
import sys
import scipy as sp
import cvxpy as cp
import pandas as pd
import numpy as np
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv1D, LSTM, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Concatenate, Reshape, Embedding, Dot
from tensorflow.keras.models import Model
sys.path.insert(0, '../src/')
%matplotlib inline

import utils
import mytools
from mytools import Timer
from mytools import Tee

/home/omid/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Parameters

In [2]:
DATA_FILE_PATH = '/home/omid/Datasets/Jeopardy/supervised_data.pk'
# DATA_FILE_PATH = '/home/omid/Datasets/Jeopardy/supervised_data_roberta.pk'
TEST_FRACTION = 0.2
RUNS = 4

# Helper functions

## General functions 

In [3]:
def reload():
    imp.reload(utils)
    imp.reload(mytools)

In [4]:
# def get_eigvec_of_laplacian(A: np.matrix) -> np.matrix:
# #     D = np.diag(np.array(np.sum(A, axis=0))[0])
# #     L = D - A
# #     return np.matrix(np.linalg.eig(L)[1])
#     n, m = A.shape
#     diags = A.sum(axis=1).flatten()
#     D = sp.sparse.spdiags(diags, [0], m, n, format='csr')
#     L = D - A
#     with sp.errstate(divide='ignore'):
#         diags_sqrt = 1.0/sp.sqrt(diags)
#     diags_sqrt[sp.isinf(diags_sqrt)] = 0
#     DH = sp.sparse.spdiags(diags_sqrt, [0], m, n, format='csr')
#     DH = DH.todense()
#     normalized_L = DH.dot(L.dot(DH))
#     return normalized_L

## Models

### Boilerplate model functions

In [5]:
def comput_error(y_train_or_validation_or_test_true, y_train_or_validation_or_test_predicted, estimation_name, error_type_str):
    """Computes the error."""
    err = 0
    for index in range(len(y_train_or_validation_or_test_true)):
        groundtruth = y_train_or_validation_or_test_true[index][estimation_name]
        predicted = y_train_or_validation_or_test_predicted[index]
        # TODO(@Omid): CHECK WHETHER THIS IS JUST AN INT THEN FIND THE MOST INFLUENTIAL PERSON FROM THE ESTIAMTED MATRIX.
        err += utils.matrix_estimation_error(
            true_matrix=groundtruth, pred_matrix=predicted, type_str=error_type_str)
    err /= len(y_train_or_validation_or_test_true)
    return err


def model_builder(
        X_train,
        y_train,
        X_test,
        y_test,
        feature_names,
        estimation_name='influence_matrix',
        error_type_str='normalized_frob_norm',
        tune_hyperparameters_by_validation=True,
        with_replication=True,
        lambdas = [0, 0.1, 1, 10, 100, 1000],
        model_func='average',
        params={'with_constraints': True, 'n_splits': 3}):
    
    # For the baseline models.
    if model_func == 'average':
        mats = []
        for i in range(len(y_train)):
            mats.append(y_train[i][estimation_name])
        y_baseline_predicted = [np.matrix(np.mean(mats, axis=0)) for _ in range(len(y_train))]
    elif model_func == 'uniform':
        y_baseline_predicted = [np.matrix(np.ones((4, 4)) * 0.25) for _ in range(len(y_train))]
    elif model_func == 'random':
        y_baseline_predicted = [np.matrix(
            utils.make_matrix_row_stochastic(
                np.random.rand(4, 4))) for _ in range(len(y_train))]
    if model_func in ['average', 'uniform', 'random']:
        y_train_average = []
        train_error = comput_error(
            y_train, y_baseline_predicted, estimation_name=estimation_name, error_type_str=error_type_str)
        test_error = comput_error(
            y_test, y_baseline_predicted, estimation_name=estimation_name, error_type_str=error_type_str)
        return train_error, test_error, _
    
    # For the proposed models.
    validation_errors = defaultdict(lambda: 0)
    if tune_hyperparameters_by_validation:
        print('{}-fold validation ...'.format(params['n_splits']))
        kf = KFold(n_splits=params['n_splits'])
        for train_index, validation_index in kf.split(X_train):
            X_train_subset, X_validation = X_train[train_index], X_train[validation_index]
            y_train_subset, y_validation = y_train[train_index], y_train[validation_index]
            if with_replication:
                X_train_subset, y_train_subset = utils.replicate_matrices_in_train_dataset_with_reordering(
                    X_train_subset, y_train_subset)
                X_train_subset = np.array(X_train_subset)
                y_train_subset = np.array(y_train_subset)
            print('Shapes of train: {}, validation: {}, test: {}.'.format(
                X_train_subset.shape, X_validation.shape, X_test.shape))
            for lambdaa in lambdas:
                validation_errors[lambdaa] += model_func(
                    X_train=X_train_subset,
                    y_train=y_train_subset,
                    X_validation_or_test=X_validation,
                    y_validation_or_test=y_validation,
                    feature_names=feature_names,
                    estimation_name=estimation_name,
                    lambdaa=lambdaa,
                    error_type_str=error_type_str,
                    params=params)[1]
        best_lambda = min(validation_errors, key=validation_errors.get)
    else:
        best_lambda = 0.1
    print('Training with the best lambda: {} on entire training set...'.format(best_lambda))
    train_error, test_error = model_func(
        X_train=X_train,
        y_train=y_train,
        X_validation_or_test=X_test,
        y_validation_or_test=y_test,
        feature_names=feature_names,
        estimation_name=estimation_name,
        lambdaa=best_lambda,
        error_type_str=error_type_str,
        params=params)
    return train_error, test_error, validation_errors

### Specific model functions

In [6]:
def convex_optimization_model_func(
        X_train,
        y_train,
        X_validation_or_test,
        y_validation_or_test,
        feature_names,
        estimation_name,
        lambdaa,
        error_type_str,
        params={'with_constraints': True}):
    
    def predict(data_element, feature_names, B, Ws, is_solved=False):
        """Defines the prediction function."""
        predicted = 0
        if is_solved:
            predicted += B.value
        else:
            predicted += B
        for feature_name in feature_names:
            if is_solved:
                W_for_this_feature = Ws[feature_name].value
            else:
                W_for_this_feature = Ws[feature_name]
            if len(data_element[feature_name].shape) == 1:
                # If it was a vector, it makes a matrix with it.
                p = data_element[feature_name]
                data_element_matrix = np.row_stack([p, p, p, p])
            else:
                # Unless it is already a matrix.
                data_element_matrix = data_element[feature_name]
            predicted += (data_element_matrix * W_for_this_feature)
        return predicted
    
    def predict_all_after_solving(X_train_or_validation_or_test, B, Ws, feature_names):
        """Predicts for all data points in the given set."""
        return [
            predict(
                data_element=data_element,
                feature_names=feature_names,
                B=B,
                Ws=Ws,
                is_solved=True) 
            for data_element in X_train_or_validation_or_test]

    # Creating variables.
    Ws = {}
    for feature_name in feature_names:
        if len(X_train[0][feature_name].shape) == 1:
            # If it was a vector, it makes a matrix with it.
            Ws[feature_name] = cp.Variable(
                len(X_train[0][feature_name]), len(X_train[0][feature_name]))
        else:
            # Unless it is already a matrix.
            Ws[feature_name] = cp.Variable(
                X_train[0][feature_name].shape[1], X_train[0][feature_name].shape[0])
    B = cp.Variable(4, 4)

    # Computing loss.
    constraints = []
    losses = 0
    for index in range(len(X_train)):
        element = X_train[index]
        estimation_groundtruth = y_train[index][estimation_name]

        # Defining the estimation function.
        estimation_predicted = predict(
            data_element=element, feature_names=feature_names, B=B, Ws=Ws, is_solved=False)

        # Defining the loss function.
        loss = cp.sum_squares(estimation_predicted - estimation_groundtruth)

        losses += loss
        if params['with_constraints']:
            constraints += [estimation_predicted >= 0]
            constraints += [cp.sum_entries(estimation_predicted, axis=1) == 1]

    # Computing regularization.
    regluarization = cp.norm1(B)
    for feature_name in feature_names:
        regluarization += cp.norm1(Ws[feature_name])

    # Solving the convex problem.
    objective = cp.Minimize(losses + lambdaa * regluarization)
    prob = cp.Problem(objective, constraints)
    try:
        result = prob.solve(solver=cp.MOSEK)
    except cp.SolverError as e:
        print('Problem was not solved for lambda: {}'.format(lambdaa))
        return sys.maxsize, sys.maxsize
    print('The status of solution was: {} and the result was: {}'.format(prob.status, result))

    # Predicting and computing trian error.
    y_train_predicted = predict_all_after_solving(
        X_train_or_validation_or_test=X_train, B=B, Ws=Ws, feature_names=feature_names)
    train_error = comput_error(
        y_train_or_validation_or_test_true=y_train,
        y_train_or_validation_or_test_predicted=y_train_predicted,
        estimation_name=estimation_name,
        error_type_str=error_type_str)
    
    # Predicting and computing validation or test error.
    y_validation_or_test_predicted = predict_all_after_solving(
        X_train_or_validation_or_test=X_validation_or_test, B=B, Ws=Ws, feature_names=feature_names)
    validation_or_test_error = comput_error(
        y_train_or_validation_or_test_true=y_validation_or_test,
        y_train_or_validation_or_test_predicted=y_validation_or_test_predicted,
        estimation_name=estimation_name,
        error_type_str=error_type_str)
    
    return train_error, validation_or_test_error

In [7]:
# def my_init(shape, dtype=None):
#     return np.ones(shape) * 0.25


def concatinated_deep_neural_network_model_func(
        X_train,
        y_train,
        X_validation_or_test,
        y_validation_or_test,
        feature_names,
        estimation_name,
        lambdaa,
        error_type_str,
        params={'n_epochs': 10, 'batch_size': 32}):
    
#     def predict(data_element, feature_names, B, Ws, is_solved=False):
#         """Defines the prediction function."""
#         return 0
    
#     def predict_all_after_solving(X_train_or_validation_or_test, B, Ws, feature_names):
#         """Predicts for all data points in the given set."""
#         return []

    flatten_X_train = []
    flatten_y_train = []
    for i in range(len(X_train)):
        features = X_train[i]
        label = y_train[i][estimation_name]            
        flatten_X_train.append(np.hstack(
            [np.array(features[feature_name].flatten())[0] for feature_name in feature_names]))
        flatten_y_train.append(np.array(label.flatten())[0])
    flatten_X_train = np.array(flatten_X_train)
    flatten_y_train = np.array(flatten_y_train)

    flatten_X_validation_or_test = []
    flatten_y_validation_or_test = []
    for i in range(len(X_validation_or_test)):
        features = X_validation_or_test[i]
        label = y_validation_or_test[i][estimation_name]
        flatten_X_validation_or_test.append(np.hstack(
            [np.array(features[feature_name].flatten())[0] for feature_name in feature_names]))
        flatten_y_validation_or_test.append(np.array(label.flatten())[0])
    flatten_X_validation_or_test = np.array(flatten_X_validation_or_test)
    flatten_y_validation_or_test = np.array(flatten_y_validation_or_test)
                              
    _, input_size = flatten_X_train.shape
    print('Input size for the neural network was: {}'.format(input_size))

    model = Sequential([
        Dense(
            units=32,
            kernel_initializer='he_normal',
            activation='relu',
            input_shape=(input_size,),
            kernel_regularizer=regularizers.l1(lambdaa),
            activity_regularizer=regularizers.l1(lambdaa)),
        Dropout(0.5),
        Dense(
            units=64,
            kernel_initializer='he_normal',
            activation='relu',
            kernel_regularizer=regularizers.l1(lambdaa),
            activity_regularizer=regularizers.l1(lambdaa)),
        Dropout(0.5),
        Dense(
            units=32,
            kernel_initializer='he_normal',
            activation='relu',
            kernel_regularizer=regularizers.l1(lambdaa),
            activity_regularizer=regularizers.l1(lambdaa)),
        Dropout(0.5),
        Dense(16, kernel_initializer='he_normal', activation='softmax')])
#         Dense(16, kernel_initializer=my_init, activation='softmax')])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.fit(flatten_X_train, flatten_y_train, epochs=params['n_epochs'], batch_size=params['batch_size'])

    # Predicting and computing trian error.
    y_train_predicted = [utils.make_matrix_row_stochastic(
        np.matrix(np.reshape(element, (4, 4)))) for element in model.predict(flatten_X_train)]
    train_error = comput_error(
        y_train_or_validation_or_test_true=y_train,
        y_train_or_validation_or_test_predicted=y_train_predicted,
        estimation_name=estimation_name,
        error_type_str=error_type_str)
                                            
    # Predicting and computing trian error.
    y_test_predicted = [utils.make_matrix_row_stochastic(
        np.matrix(np.reshape(element, (4, 4)))) for element in model.predict(flatten_X_validation_or_test)]
    validation_or_test_error = comput_error(
        y_train_or_validation_or_test_true=y_test,
        y_train_or_validation_or_test_predicted=y_test_predicted,
        estimation_name=estimation_name,
        error_type_str=error_type_str)

    return train_error, validation_or_test_error

# Loading the data

In [8]:
data = utils.load_it(DATA_FILE_PATH)
print(len(data['X']))

264


### Just to see how much headroom there is:

In [9]:
mats = []
for i in range(len(data['y'])):
    mats.append(data['y'][i]['influence_matrix'])
np.mean(mats, axis=0)

array([[0.25784649, 0.25013211, 0.25917224, 0.23284916],
       [0.21997335, 0.3244428 , 0.22187468, 0.23370918],
       [0.21050033, 0.25039752, 0.31083892, 0.22826324],
       [0.25180346, 0.23839655, 0.24293584, 0.26686416]])

In [10]:
np.mean(mats, axis=0)

array([[0.25784649, 0.25013211, 0.25917224, 0.23284916],
       [0.21997335, 0.3244428 , 0.22187468, 0.23370918],
       [0.21050033, 0.25039752, 0.31083892, 0.22826324],
       [0.25180346, 0.23839655, 0.24293584, 0.26686416]])

In [11]:
np.std(mats, axis=0)

array([[0.11677549, 0.08129962, 0.08962429, 0.07536874],
       [0.08929333, 0.16587926, 0.08335618, 0.10916787],
       [0.08963767, 0.12213777, 0.16865608, 0.09193362],
       [0.09597464, 0.06509444, 0.05915974, 0.08748726]])

#### Message: It seems the std is small, and average is close to 0.25 everywhere but main diagonal which is slightly larger (due to having selfish people).

# Main body

In [12]:
data['X'][0].keys()

dict_keys(['sentiment', 'individual_performance', 'content_embedding_matrix', 'reply_duration', 'individual_performance_hardness_weighted', 'emotion_dominance', 'first_influence_matrix', 'emotion_arousal', 'emotion_valence', 'average_of_previous_influence_matrices'])

In [13]:
LAMBDAS = [0, 0.1, 1, 10, 100, 1000]
WITH_REPLICATION = True
ERROR_TYPE_STRS = ['normalized_frob_norm', 'mse', 'neg_corr', 'cosine_dist']
TUNE_HYPERPARAMETERS_BY_VALIDATION = True
SELECTED_MODEL_FUNCS = ['average', 'uniform', 'random', convex_optimization_model_func, concatinated_deep_neural_network_model_func]
FEATURE_NAMES_SET = [['individual_performance'], ['first_influence_matrix'], ['reply_duration'], ['emotion_dominance'], ['average_of_previous_influence_matrices']]  # ['content_embedding_matrix'],
ESTIMATION_NAME = 'influence_matrix' #'most_influentials'

TO_WRITE_LOGS_ON_FILE = True

## Estimation comparison pipeline

In [ ]:
if TO_WRITE_LOGS_ON_FILE:
    f = open('Outputs/output{}.txt'.format(str(datetime.datetime.now())), 'w')
    sys.stdout = Tee(sys.stdout, f)


with Timer():
    train_errors_in_runs = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    test_errors_in_runs = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    validation_errors_in_runs = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    for run in range(RUNS):
        print('Run', run + 1, '=>>>')
        X_train, X_test, y_train, y_test = train_test_split(
            np.array(data['X']), np.array(data['y']), test_size=TEST_FRACTION)
        for feature_names in FEATURE_NAMES_SET:
            print('\tFeatures: ', feature_names, '...')
            for selected_model_func in SELECTED_MODEL_FUNCS:
                if isinstance(selected_model_func, str):
                    selected_model_func_str = selected_model_func
                else:
                    selected_model_func_str = selected_model_func.__name__
                print('\t\tModel: ', selected_model_func_str, '...')
                for error_type_str in ERROR_TYPE_STRS:
                    print('\t\t\tError type: ', error_type_str)
                    train_error = 0
                    test_error = 0
                    validation_errors = 0
                    try:
                        train_error, test_error, validation_errors = model_builder(
                            X_train=X_train,
                            y_train=y_train,
                            X_test=X_test,
                            y_test=y_test,
                            feature_names=feature_names,
                            estimation_name=ESTIMATION_NAME,
                            error_type_str=error_type_str,
                            tune_hyperparameters_by_validation=TUNE_HYPERPARAMETERS_BY_VALIDATION,
                            with_replication=WITH_REPLICATION,
                            lambdas=LAMBDAS,
                            model_func=selected_model_func,
                            params={'with_constraints': True, 'n_splits': 3, 'n_epochs': 10, 'batch_size': 32})
                    except Exception as e:
                        print('An exception was raised:')
                        print(type(e.args))
                        print(e)
                        print('\n')
                        continue
                    key_str = error_type_str + ':' + str(feature_names) + ':' + selected_model_func_str
                    train_errors_in_runs[error_type_str][str(feature_names)][selected_model_func_str].append(
                        train_error)
                    test_errors_in_runs[error_type_str][str(feature_names)][selected_model_func_str].append(
                        test_error)
                    validation_errors_in_runs[error_type_str][str(feature_names)][selected_model_func_str].append(
                        validation_errors)
        utils.save_it(
            obj=train_errors_in_runs,
            file_path='Outputs/train_errors{}.pkl'.format(run+1),
            verbose=True)
        utils.save_it(
            obj=test_errors_in_runs,
            file_path='Outputs/test_errors{}.pkl'.format(run+1),
            verbose=True)
        utils.save_it(
            obj=validation_errors_in_runs,
            file_path='Outputs/validation_errors{}.pkl'.format(run+1),
            verbose=True)
    print('ALL DONE SUCCESSFULLY.')

Run 1 =>>>
	Features:  ['individual_performance'] ...
		Model:  average ...
			Error type:  normalized_frob_norm
			Error type:  mse
			Error type:  neg_corr
			Error type:  cosine_dist
		Model:  uniform ...
			Error type:  normalized_frob_norm
			Error type:  mse
			Error type:  neg_corr
			Error type:  cosine_dist
		Model:  random ...
			Error type:  normalized_frob_norm
			Error type:  mse
			Error type:  neg_corr


/home/omid/.local/lib/python3.5/site-packages/scipy/stats/stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


			Error type:  cosine_dist
		Model:  convex_optimization_model_func ...
			Error type:  normalized_frob_norm
3-fold validation ...
Shapes of train: (3360,), validation: (71,), test: (53,).
The status of solution was: optimal and the result was: 561.8616024871658
The status of solution was: optimal and the result was: 562.3514862293198
The status of solution was: optimal and the result was: 566.7486096101338
The status of solution was: optimal and the result was: 609.5674330856258
The status of solution was: optimal and the result was: 977.5141969551998
The status of solution was: optimal and the result was: 4577.514163516418
Shapes of train: (3384,), validation: (70,), test: (53,).
The status of solution was: optimal and the result was: 561.9641916370056
The status of solution was: optimal and the result was: 562.4340545500061
The status of solution was: optimal and the result was: 566.6508565172455
The status of solution was: optimal and the result was: 607.6645282759258
The status o

3360/3360 [==============================] - ETA: 0s - loss: 0.7405 - accuracy: 0.01 - ETA: 0s - loss: 0.7427 - accuracy: 0.01 - ETA: 0s - loss: 0.7423 - accuracy: 0.01 - 0s 40us/sample - loss: 0.7421 - accuracy: 0.0112
Epoch 5/10
3360/3360 [==============================] - ETA: 0s - loss: 0.7413 - accuracy: 0.01 - ETA: 0s - loss: 0.7415 - accuracy: 0.01 - ETA: 0s - loss: 0.7415 - accuracy: 0.01 - 0s 39us/sample - loss: 0.7413 - accuracy: 0.0112
Epoch 6/10
3360/3360 [==============================] - ETA: 0s - loss: 0.7415 - accuracy: 0.01 - ETA: 0s - loss: 0.7411 - accuracy: 0.01 - ETA: 0s - loss: 0.7411 - accuracy: 0.01 - 0s 37us/sample - loss: 0.7410 - accuracy: 0.0112
Epoch 7/10
3360/3360 [==============================] - ETA: 0s - loss: 0.7404 - accuracy: 0.01 - ETA: 0s - loss: 0.7409 - accuracy: 0.01 - ETA: 0s - loss: 0.7409 - accuracy: 0.01 - 0s 36us/sample - loss: 0.7408 - accuracy: 0.0112
Epoch 8/10
3360/3360 [==============================] - ETA: 0s - loss: 0.7406 - accura

Epoch 7/10
3360/3360 [==============================] - ETA: 0s - loss: 295.3095 - accuracy: 0.00 - ETA: 0s - loss: 292.0847 - accuracy: 0.01 - ETA: 0s - loss: 286.4904 - accuracy: 0.01 - 0s 40us/sample - loss: 283.0161 - accuracy: 0.0112
Epoch 8/10
3360/3360 [==============================] - ETA: 0s - loss: 270.8125 - accuracy: 0.01 - ETA: 0s - loss: 263.8441 - accuracy: 0.01 - ETA: 0s - loss: 259.1075 - accuracy: 0.01 - 0s 41us/sample - loss: 256.2216 - accuracy: 0.0112
Epoch 9/10
3360/3360 [==============================] - ETA: 0s - loss: 245.2654 - accuracy: 0.01 - ETA: 0s - loss: 239.4489 - accuracy: 0.01 - ETA: 0s - loss: 234.7289 - accuracy: 0.01 - 0s 39us/sample - loss: 232.3491 - accuracy: 0.0112
Epoch 10/10
3360/3360 [==============================] - ETA: 0s - loss: 220.3433 - accuracy: 0.01 - ETA: 0s - loss: 216.0200 - accuracy: 0.01 - ETA: 0s - loss: 211.5536 - accuracy: 0.01 - 0s 39us/sample - loss: 209.2114 - accuracy: 0.0112
Input size for the neural network was: 1
Tr

...

3384/3384 [==============================] - ETA: 0s - loss: 2.1194 - accuracy: 0.01 - ETA: 0s - loss: 1.8663 - accuracy: 0.00 - ETA: 0s - loss: 1.6959 - accuracy: 0.01 - ETA: 0s - loss: 1.6076 - accuracy: 0.01 - 0s 48us/sample - loss: 1.6019 - accuracy: 0.0115
Epoch 9/10
3384/3384 [==============================] - ETA: 0s - loss: 1.3834 - accuracy: 0.01 - ETA: 0s - loss: 1.3420 - accuracy: 0.01 - ETA: 0s - loss: 1.3305 - accuracy: 0.01 - ETA: 0s - loss: 1.3253 - accuracy: 0.01 - 0s 51us/sample - loss: 1.3239 - accuracy: 0.0115
Epoch 10/10
3384/3384 [==============================] - ETA: 0s - loss: 1.3132 - accuracy: 0.00 - ETA: 0s - loss: 1.3142 - accuracy: 0.01 - ETA: 0s - loss: 1.3139 - accuracy: 0.01 - ETA: 0s - loss: 1.3139 - accuracy: 0.01 - 0s 48us/sample - loss: 1.3139 - accuracy: 0.0115
Input size for the neural network was: 16
Train on 3384 samples
Epoch 1/10
3384/3384 [==============================] - ETA: 1:08 - loss: 8890.7441 - accuracy: 0.009 - ETA: 1s - loss: 8214.57

3384/3384 [==============================] - ETA: 0s - loss: 6018.6709 - accuracy: 0.017 - ETA: 0s - loss: 5394.7345 - accuracy: 0.010 - ETA: 0s - loss: 4827.8183 - accuracy: 0.011 - ETA: 0s - loss: 4351.2732 - accuracy: 0.011 - 0s 55us/sample - loss: 4100.7474 - accuracy: 0.0115
Epoch 8/10
3384/3384 [==============================] - ETA: 0s - loss: 2659.7983 - accuracy: 0.015 - ETA: 0s - loss: 2400.8369 - accuracy: 0.009 - ETA: 0s - loss: 2163.7021 - accuracy: 0.011 - ETA: 0s - loss: 1967.0209 - accuracy: 0.011 - 0s 52us/sample - loss: 1900.7523 - accuracy: 0.0115
Epoch 9/10
3384/3384 [==============================] - ETA: 0s - loss: 1315.1349 - accuracy: 0.002 - ETA: 0s - loss: 1204.8491 - accuracy: 0.009 - ETA: 0s - loss: 1113.1680 - accuracy: 0.010 - ETA: 0s - loss: 1046.1125 - accuracy: 0.011 - 0s 55us/sample - loss: 1009.2181 - accuracy: 0.0115
Epoch 10/10
3384/3384 [==============================] - ETA: 0s - loss: 798.3506 - accuracy: 0.00 - ETA: 0s - loss: 751.9523 - accurac

3384/3384 [==============================] - ETA: 0s - loss: 2.5797 - accuracy: 0.00 - ETA: 0s - loss: 2.3158 - accuracy: 0.01 - ETA: 0s - loss: 2.1503 - accuracy: 0.01 - 0s 44us/sample - loss: 2.0390 - accuracy: 0.0102
Epoch 9/10
3384/3384 [==============================] - ETA: 0s - loss: 1.7063 - accuracy: 0.01 - ETA: 0s - loss: 1.6208 - accuracy: 0.01 - ETA: 0s - loss: 1.5511 - accuracy: 0.01 - 0s 43us/sample - loss: 1.5025 - accuracy: 0.0102
Epoch 10/10
3384/3384 [==============================] - ETA: 0s - loss: 1.3632 - accuracy: 0.01 - ETA: 0s - loss: 1.3561 - accuracy: 0.00 - ETA: 0s - loss: 1.3412 - accuracy: 0.00 - 0s 43us/sample - loss: 1.3339 - accuracy: 0.0102
Input size for the neural network was: 16
Train on 3384 samples
Epoch 1/10
3384/3384 [==============================] - ETA: 1:08 - loss: 9002.8975 - accuracy: 0.005 - ETA: 1s - loss: 8279.3099 - accuracy: 0.0088  - ETA: 0s - loss: 7516.0506 - accuracy: 0.009 - ETA: 0s - loss: 6919.6900 - accuracy: 0.009 - 1s 245us/

3384/3384 [==============================] - ETA: 0s - loss: 1006.3945 - accuracy: 0.009 - ETA: 0s - loss: 886.5155 - accuracy: 0.010 - ETA: 0s - loss: 805.9265 - accuracy: 0.01 - ETA: 0s - loss: 747.3917 - accuracy: 0.01 - 0s 47us/sample - loss: 745.2043 - accuracy: 0.0102
Epoch 10/10
3384/3384 [==============================] - ETA: 0s - loss: 612.4388 - accuracy: 0.01 - ETA: 0s - loss: 601.9920 - accuracy: 0.00 - ETA: 0s - loss: 587.8395 - accuracy: 0.01 - 0s 44us/sample - loss: 580.9781 - accuracy: 0.0102
Training with the best lambda: 0.1 on entire training set...
Input size for the neural network was: 16
Train on 211 samples
Epoch 1/10
211/211 [==============================] - ETA: 4s - loss: 89.7942 - accuracy: 0.003 - 1s 4ms/sample - loss: 88.5073 - accuracy: 0.0104
Epoch 2/10
211/211 [==============================] - ETA: 0s - loss: 86.2157 - accuracy: 0.002 - 0s 118us/sample - loss: 85.1000 - accuracy: 0.0110
Epoch 3/10
211/211 [==============================] - ETA: 0s - l

The status of solution was: optimal and the result was: 4652.365085025896
Training with the best lambda: 1000 on entire training set...
The status of solution was: optimal and the result was: 4038.89228187703
			Error type:  cosine_dist
3-fold validation ...
Shapes of train: (3360,), validation: (71,), test: (53,).
The status of solution was: optimal and the result was: 579.920443368662
The status of solution was: optimal and the result was: 580.4024009198477
The status of solution was: optimal and the result was: 584.7276466673641
The status of solution was: optimal and the result was: 626.7757903568636
The status of solution was: optimal and the result was: 992.3764470916208
The status of solution was: optimal and the result was: 4592.376337400068
Shapes of train: (3384,), validation: (70,), test: (53,).
The status of solution was: optimal and the result was: 672.603761958405
The status of solution was: optimal and the result was: 673.0750010390188
The status of solution was: optimal

## Visualization of the results

In [ ]:
# train_errors_in_runs = utils.load_it('Outputs/train_errors2_old.dill')
# test_errors_in_runs = utils.load_it('Outputs/test_errors2_old.dill')

In [ ]:
for error_type_str, features_dt in train_errors_in_runs.items():
    print(error_type_str, ':')
    for feature_set, models_dt in features_dt.items():
        print(feature_set, ':')
        for model_str, errors in models_dt.items():
            print(model_str, ':')
            print(np.mean(errors), '+-', np.std(errors), '\n')

In [ ]:
for error_type_str, features_dt in test_errors_in_runs.items():
    print(error_type_str, ':')
    for feature_set, models_dt in features_dt.items():
        print('\t', feature_set, ':')
        for model_str, errors in models_dt.items():
            print('\t\t', model_str, ':')
            print('\t\t', round(np.mean(errors), 4), '+-', round(np.std(errors), 4), '\n')

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     np.array(data['X']), np.array(data['y']), test_size=TEST_FRACTION)

In [ ]:
# a, b, c = model_builder(
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test,
#     feature_names=['average_of_previous_influence_matrices'],
#     estimation_name=ESTIMATION_NAME,
#     error_type_str='normalized_frob_norm',
#     tune_hyperparameters_by_validation=True,
#     with_replication=False,
#     lambdas=LAMBDAS,
#     model_func=convex_optimization_model_func,
#     params={'with_constraints': False, 'n_splits': 3, 'n_epochs': 10, 'batch_size': 32})

In [ ]:
# a

In [ ]:
# b

In [ ]:
# c